In [12]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
import pandas as pd
import tensorflow as tf
import numpy as np
import gensim
import gensim.models.keyedvectors as word2vec
from sklearn.metrics.pairwise import cosine_similarity
import pickle

In [14]:
from parse_data import parseData

In [15]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [16]:
baseline_f = pd.read_csv('./www2018-table/feature/features.csv')

In [ ]:
w2v_gn_model = word2vec.KeyedVectors.load_word2vec_format("./GoogleNews-vectors-negative300.bin", binary=True)  

In [ ]:
# w2v_gn_model.similarity('computer','ram')

In [ ]:
# w2v_gn_model.get_vector('computer')

In [7]:
rdv2vec_model = gensim.models.Word2Vec.load('./DB2Vec_sg_200_5_5_15_2_500')

/home/vibhav/bar/virtualenv/lib/python3.5/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
/home/vibhav/bar/virtualenv/lib/python3.5/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [ ]:
# rdv2vec_model.wv.similarity('dbr:England','dbr:United_Kingdom')

In [ ]:
rdv2vec_model.wv['dbr:SNCF_TGV_Atlantique']

In [17]:
tables_path = '/home/vibhav/table2vec/tables_redi2_1/'

def get_table_path(table_id):
    return tables_path + 're_tables-' + table_id.split('-')[1] + '.json'

In [18]:
baseline_f['table_path'] = baseline_f.table_id.apply(get_table_path)

In [19]:
baseline_f['parsedTable'] = baseline_f.apply(lambda x: parseData(x['table_path'],x['table_id']).split(' '),axis=1)

/home/vibhav/table2vec/tables_redi2_1/re_tables-1579.json
/home/vibhav/table2vec/tables_redi2_1/re_tables-1578.json
/home/vibhav/table2vec/tables_redi2_1/re_tables-1211.json
/home/vibhav/table2vec/tables_redi2_1/re_tables-1210.json
/home/vibhav/table2vec/tables_redi2_1/re_tables-1411.json
/home/vibhav/table2vec/tables_redi2_1/re_tables-1410.json


In [ ]:
def get_w2v_embd(sentence):
    embd = []
    for word in sentence:
        try:
            embd.append(w2v_gn_model.get_vector(word))
        except Exception as e:
            embd.append(w2v_gn_model.get_vector('UNK'))
    embd = np.array(embd)
    return embd

In [ ]:
baseline_f['w2v_embd_table'] = baseline_f.parsedTable.apply(get_w2v_embd)

In [9]:
baseline_f.head()

,query_id,query,table_id,row,col,nul,in_link,out_link,pgcount,tImp,tPF,leftColhits,SecColhits,bodyhits,PMI,qInPgTitle,qInTableTitle,yRank,csr_score,idf1,idf2,idf3,idf4,idf5,idf6,max,sum,avg,sim,emax,esum,eavg,esim,cmax,csum,cavg,csim,remax,resum,reavg,resim,query_l,rel,table_path,parsedTable
0,1,world interest rates Table,table-0875-680,8,2,0,31,21,51438,1.0,0.000260,0,0,0,0.000000,0.333333,0.222222,100,7.467415e-10,29.627861,24.135644,27.100638,31.219261,27.359181,27.100638,1.000000,9.266177,0.110312,0.676819,0.952804,89.707105,0.815519,0.971854,0.666667,5.291894,0.048108,0.354686,0.241209,3.716354,0.033785,0.281130,4,0,/home/vibhav/table2vec/tables_redi2_1/re_table...,"[experian, score, expected, annual, loss, rate..."
1,1,world interest rates Table,table-1020-619,4,3,0,18,0,324,1.0,0.000786,0,0,0,-0.231049,0.000000,0.285714,100,8.663725e-10,29.627861,24.135644,27.100638,31.219261,27.359181,27.100638,1.000000,3.038269,0.108510,0.863125,1.000000,83.375200,0.757956,0.983893,1.000000,11.116121,0.101056,0.718895,1.000000,8.075247,0.073411,0.710250,4,0,/home/vibhav/table2vec/tables_redi2_1/re_table...,"[annual, inflation, rates, aug, sep, silver, u..."
2,1,world interest rates Table,table-0288-531,3,5,0,23,22,26419,0.5,0.000158,0,0,0,-0.069315,0.000000,0.000000,100,2.142316e-10,29.627861,24.135644,27.100638,31.219261,27.359181,27.100638,0.234136,1.747577,0.043689,0.206247,0.841653,57.442424,0.638249,0.753198,0.000000,0.000000,0.000000,0.000000,0.067373,0.365818,0.003326,0.033680,4,0,/home/vibhav/table2vec/tables_redi2_1/re_table...,"[regret, interest, rates, rise, static, rates,..."
3,1,world interest rates Table,table-0288-530,4,5,1,23,22,26419,0.5,0.000211,0,0,0,-0.069315,0.000000,0.000000,100,1.888429e-10,29.627861,24.135644,27.100638,31.219261,27.359181,27.100638,0.234136,1.641433,0.041036,0.252283,0.841653,57.442424,0.638249,0.753198,0.000000,0.000000,0.000000,0.000000,0.067373,0.365818,0.003326,0.033680,4,0,/home/vibhav/table2vec/tables_redi2_1/re_table...,"[return, interest, rates, rise, static, rates,..."
4,1,world interest rates Table,table-1000-57,2,2,0,38,1,2268,1.0,0.000130,0,0,0,0.000000,0.000000,0.500000,100,7.673589e-10,29.627861,24.135644,27.100638,31.219261,27.359181,27.100638,1.000000,4.982802,0.113245,0.700773,0.932809,89.552347,0.814112,0.954395,1.000000,10.147388,0.092249,0.372667,0.226134,4.564622,0.041497,0.279899,4,0,/home/vibhav/table2vec/tables_redi2_1/re_table...,"[security, nominal, interest, rates, chakravar..."


In [20]:
semantic_f = baseline_f.loc[:,['query_id','query','table_path','table_id','parsedTable','rel']]
baseline_f.drop(columns=['table_path','parsedTable'],inplace=True)

In [ ]:
semantic_f['w2v_embd_query'] = semantic_f['query'].apply(lambda x: get_w2v_embd(x.split(" ")))

In [8]:
def early_fusion(table,query):
    a = np.average(table,axis=0).reshape(1,-1)
    b = np.average(query,axis=0).reshape(1,-1)
    sim = cosine_similarity(a,b)
    return sim.reshape(-1)[0]

In [ ]:
semantic_f['w2v_early_fusion'] = semantic_f.apply(lambda x: early_fusion(x['w2v_embd_table'],x['w2v_embd_query']),axis=1)

In [ ]:
# semantic_f[semantic_f.w2v_early_fusion < 0.3].rel.value_counts()
# semantic_f[semantic_f.w2v_late_fusion_avg < 0.3].shape

In [9]:
def late_fusion(table,query):
    s = []
    for i in query:
        for j in table:
            sim = cosine_similarity(i.reshape(1,-1),j.reshape(1,-1))
            s.append(sim)
    s = np.array(s).reshape(-1)
    return s

In [ ]:
semantic_f['w2v_late_fusion'] = semantic_f.apply(lambda x: late_fusion(x['w2v_embd_table'],x['w2v_embd_query']),axis=1)

In [ ]:
semantic_f['w2v_late_fusion_max'] = semantic_f.w2v_late_fusion.apply(np.max)
semantic_f['w2v_late_fusion_avg'] = semantic_f.w2v_late_fusion.apply(np.average)
semantic_f['w2v_late_fusion_sum'] = semantic_f.w2v_late_fusion.apply(np.sum)

In [ ]:
semantic_f.head()

In [10]:
from get_entities import getEntities

In [11]:
query_entities = {}
for i in semantic_f['query'].unique():
    query_entities[i] = getEntities(i)

semantic_f['query_entites'] = semantic_f['query'].apply(lambda x: query_entities[x])

NameError: name 'semantic_f' is not defined

In [ ]:
semantic_f.query_entites.iloc[1]

In [10]:
df_1 = pd.read_csv('./semantic_f_w2v_entities_1.csv')
df_2 = pd.read_csv('./semantic_f_w2v_entities_2.csv')
df_3 = pd.read_csv('./semantic_f_w2v_entities_3.csv')
df_4 = pd.read_csv('./semantic_f_w2v_entities_4.csv')

In [11]:
df = pd.concat([df_1,df_2,df_3,df_4])

In [12]:
import ast
df['table_entities'] = df.table_entities.apply(lambda x: ast.literal_eval(x))

In [13]:
from pandarallel import pandarallel

In [14]:
pandarallel.initialize(progress_bar=True,nb_workers=15,shm_size_mb=2500)


New pandarallel memory created - Size: 2500 MB
Pandarallel will run on 15 workers


In [15]:
def get_g2v_embd(entities):
    embd = []
    for en in entities:
        word = 'dbr:' + en
        try:
            embd.append(rdv2vec_model.wv[word])
        except Exception as e:
            embd.append(rdv2vec_model.wv['dbr:UNK'])
    return np.array(embd)

In [16]:
df['g2v_embd_table'] = df.table_entities.apply(get_g2v_embd)

In [17]:
df['g2v_embd_query'] = df.query_entites.apply(get_g2v_embd)

In [18]:
df['g2v_early_fusion'] = df.apply(lambda x: early_fusion(x['g2v_embd_table'],x['g2v_embd_query']),axis=1)

In [19]:
df['g2v_late_fusion'] = df.parallel_apply(lambda x: late_fusion(x['g2v_embd_table'],x['g2v_embd_query']),axis=1)

In [21]:
df['g2v_late_fusion_max'] = df.g2v_late_fusion.apply(np.max)
df['g2v_late_fusion_avg'] = df.g2v_late_fusion.apply(np.average)
df['g2v_late_fusion_sum'] = df.g2v_late_fusion.apply(np.sum)

In [22]:
df.head()

,query_id,query,table_path,table_id,parsedTable,w2v_embd_table,rel,w2v_embd_query,w2v_early_fusion,w2v_late_fusion,w2v_late_fusion_max,w2v_late_fusion_avg,w2v_late_fusion_sum,query_entites,table_entities,g2v_embd_table,g2v_embd_query,g2v_early_fusion,g2v_late_fusion,g2v_late_fusion_max,g2v_late_fusion_avg,g2v_late_fusion_sum
0,1,world interest rates Table,/home/vibhav/table2vec/tables_redi2_1/re_table...,table-0875-680,"['experian', 'score', 'expected', 'annual', 'l...",[[-0.09472656 -0.51171875 0.05810547 ... 0.3...,0,[[-6.3964844e-02 6.8359375e-02 2.2460938e-01...,0.572375,[ 0.07764402 0.04398614 0.062119 0.131361...,1.0,0.132950,18.081264,{'3': {'entity': '<dbpedia:Permanent_interest_...,"[Cash_advance, Subprime_lending, Credit_card_b...","[[-0.105519615, 0.16365984, -0.10385874, -0.01...","[[-0.16757718, 0.09818904, -0.15539637, 0.0454...",0.630084,"[0.8126399, 0.5829352, 0.8141425, 0.84502095, ...",0.857562,0.585746,10747.270508
1,1,world interest rates Table,/home/vibhav/table2vec/tables_redi2_1/re_table...,table-1020-619,"['annual', 'inflation', 'rates', 'aug', 'sep',...",[[ 0.0390625 0.01586914 -0.07373047 ... 0.1...,0,[[-6.3964844e-02 6.8359375e-02 2.2460938e-01...,0.681248,[ 0.13136129 0.06132873 0.00324154 0.019790...,1.0,0.160487,10.271143,{'3': {'entity': '<dbpedia:Permanent_interest_...,"[Ultra-Tories, Zero_interest-rate_policy, Inte...","[[-0.5667573, 0.059294734, -0.35778925, 0.0050...","[[-0.16757718, 0.09818904, -0.15539637, 0.0454...",0.769993,"[0.5664706, 0.6671674, 0.63320464, 0.8086046, ...",0.899880,0.620371,12417.335938
2,1,world interest rates Table,/home/vibhav/table2vec/tables_redi2_1/re_table...,table-0288-531,"['regret', 'interest', 'rates', 'rise', 'stati...",[[ 1.9287109e-02 7.5195312e-02 3.4179688e-01...,0,[[-6.3964844e-02 6.8359375e-02 2.2460938e-01...,0.606677,[ 0.09074079 0.07134737 0.00324154 0.122679...,1.0,0.145743,12.825369,{'3': {'entity': '<dbpedia:Permanent_interest_...,"[Expected_utility_hypothesis, Loss_function, B...","[[-0.01618436, 0.36752364, -0.41611332, -0.236...","[[-0.16757718, 0.09818904, -0.15539637, 0.0454...",0.879135,"[0.5724195, 0.57001764, 0.9345857, 0.87030643,...",1.000000,0.646446,11321.862305
3,1,world interest rates Table,/home/vibhav/table2vec/tables_redi2_1/re_table...,table-0288-530,"['return', 'interest', 'rates', 'rise', 'stati...",[[ 5.1269531e-02 1.2597656e-01 -1.3769531e-01...,0,[[-6.3964844e-02 6.8359375e-02 2.2460938e-01...,0.617735,[ 0.0134912 0.07134737 0.00324154 0.122679...,1.0,0.140305,13.469313,{'3': {'entity': '<dbpedia:Permanent_interest_...,"[Allan_Saint-Maximin, Maximin_Coia, Saint-Maxi...","[[-0.28710952, 0.19961576, -0.22583863, 0.0329...","[[-0.16757718, 0.09818904, -0.15539637, 0.0454...",0.866539,"[0.77406514, 0.61262506, 0.56347716, 0.6929066...",1.000000,0.638918,11722.868164
4,1,world interest rates Table,/home/vibhav/table2vec/tables_redi2_1/re_table...,table-1000-57,"['security', 'nominal', 'interest', 'rates', '...",[[ 1.8920898e-02 1.8554688e-01 4.3945312e-02...,0,[[-6.3964844e-02 6.8359375e-02 2.2460938e-01...,0.586386,[ 9.76208225e-02 4.97327670e-02 7.13473707e-...,1.0,0.139647,18.433437,{'3': {'entity': '<dbpedia:Permanent_interest_...,"[Quantitative_easing, Inflation_derivative, El...","[[-0.08840087, 0.19392557, -0.24297218, -0.365...","[[-0.16757718, 0.09818904, -0.15539637, 0.0454...",0.728584,"[0.5647532, 0.72312397, 0.8510286, 0.67512596,...",1.000000,0.633952,11103.036133


In [24]:
df.to_csv('./semantic_f_w2v_g2v.csv',index=False)

## Model Building

In [6]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [7]:
semantic_f_ = pd.read_csv('./semantic_f_w2v_g2v.csv')

In [8]:
semantic_f_.head()

,query_id,query,table_path,table_id,parsedTable,w2v_embd_table,rel,w2v_embd_query,w2v_early_fusion,w2v_late_fusion,w2v_late_fusion_max,w2v_late_fusion_avg,w2v_late_fusion_sum,query_entites,table_entities,g2v_embd_table,g2v_embd_query,g2v_early_fusion,g2v_late_fusion,g2v_late_fusion_max,g2v_late_fusion_avg,g2v_late_fusion_sum
0,1,world interest rates Table,/home/vibhav/table2vec/tables_redi2_1/re_table...,table-0875-680,"['experian', 'score', 'expected', 'annual', 'l...",[[-0.09472656 -0.51171875 0.05810547 ... 0.3...,0,[[-6.3964844e-02 6.8359375e-02 2.2460938e-01...,0.572375,[ 0.07764402 0.04398614 0.062119 0.131361...,1.0,0.132950,18.081264,{'3': {'entity': '<dbpedia:Permanent_interest_...,"['Cash_advance', 'Subprime_lending', 'Credit_c...",[[-0.10551962 0.16365984 -0.10385874 ... 0.0...,[[-0.16757718 0.09818904 -0.15539637 ... 0.0...,0.630084,[0.8126399 0.5829352 0.8141425 ... 0.798044...,0.857562,0.585746,10747.270508
1,1,world interest rates Table,/home/vibhav/table2vec/tables_redi2_1/re_table...,table-1020-619,"['annual', 'inflation', 'rates', 'aug', 'sep',...",[[ 0.0390625 0.01586914 -0.07373047 ... 0.1...,0,[[-6.3964844e-02 6.8359375e-02 2.2460938e-01...,0.681248,[ 0.13136129 0.06132873 0.00324154 0.019790...,1.0,0.160487,10.271143,{'3': {'entity': '<dbpedia:Permanent_interest_...,"['Ultra-Tories', 'Zero_interest-rate_policy', ...",[[-0.5667573 0.05929473 -0.35778925 ... -0.3...,[[-0.16757718 0.09818904 -0.15539637 ... 0.0...,0.769993,[0.5664706 0.6671674 0.63320464 ... 0.680116...,0.899880,0.620371,12417.335938
2,1,world interest rates Table,/home/vibhav/table2vec/tables_redi2_1/re_table...,table-0288-531,"['regret', 'interest', 'rates', 'rise', 'stati...",[[ 1.9287109e-02 7.5195312e-02 3.4179688e-01...,0,[[-6.3964844e-02 6.8359375e-02 2.2460938e-01...,0.606677,[ 0.09074079 0.07134737 0.00324154 0.122679...,1.0,0.145743,12.825369,{'3': {'entity': '<dbpedia:Permanent_interest_...,"['Expected_utility_hypothesis', 'Loss_function...",[[-0.01618436 0.36752364 -0.41611332 ... -0.2...,[[-0.16757718 0.09818904 -0.15539637 ... 0.0...,0.879135,[0.5724195 0.57001764 0.9345857 ... 0.456453...,1.000000,0.646446,11321.862305
3,1,world interest rates Table,/home/vibhav/table2vec/tables_redi2_1/re_table...,table-0288-530,"['return', 'interest', 'rates', 'rise', 'stati...",[[ 5.1269531e-02 1.2597656e-01 -1.3769531e-01...,0,[[-6.3964844e-02 6.8359375e-02 2.2460938e-01...,0.617735,[ 0.0134912 0.07134737 0.00324154 0.122679...,1.0,0.140305,13.469313,{'3': {'entity': '<dbpedia:Permanent_interest_...,"['Allan_Saint-Maximin', 'Maximin_Coia', 'Saint...",[[-0.28710952 0.19961576 -0.22583863 ... 0.2...,[[-0.16757718 0.09818904 -0.15539637 ... 0.0...,0.866539,[0.77406514 0.61262506 0.56347716 ... 0.734163...,1.000000,0.638918,11722.868164
4,1,world interest rates Table,/home/vibhav/table2vec/tables_redi2_1/re_table...,table-1000-57,"['security', 'nominal', 'interest', 'rates', '...",[[ 1.8920898e-02 1.8554688e-01 4.3945312e-02...,0,[[-6.3964844e-02 6.8359375e-02 2.2460938e-01...,0.586386,[ 9.76208225e-02 4.97327670e-02 7.13473707e-...,1.0,0.139647,18.433437,{'3': {'entity': '<dbpedia:Permanent_interest_...,"['Quantitative_easing', 'Inflation_derivative'...",[[-0.08840087 0.19392557 -0.24297218 ... -0.3...,[[-0.16757718 0.09818904 -0.15539637 ... 0.0...,0.728584,[0.5647532 0.72312397 0.8510286 ... 0.741340...,1.000000,0.633952,11103.036133


In [14]:
baseline_f.columns

Index(['query_id', 'query', 'table_id', 'row', 'col', 'nul', 'in_link', 'out_link', 'pgcount', 'tImp', 'tPF', 'leftColhits', 'SecColhits', 'bodyhits', 'PMI', 'qInPgTitle', 'qInTableTitle', 'yRank', 'csr_score', 'idf1', 'idf2', 'idf3', 'idf4', 'idf5', 'idf6', 'max', 'sum', 'avg', 'sim', 'emax', 'esum', 'eavg', 'esim', 'cmax', 'csum', 'cavg', 'csim', 'remax', 'resum', 'reavg', 'resim', 'query_l', 'rel'], dtype='object')

In [15]:
x_bf = ['row', 'col', 'nul', 'in_link', 'out_link', 'pgcount', 'tImp', 'tPF', 'leftColhits', 'SecColhits', 'bodyhits', 'PMI', 'qInPgTitle', 'qInTableTitle', 'yRank', 'csr_score', 'idf1', 'idf2', 'idf3', 'idf4', 'idf5', 'idf6', 'max', 'sum', 'avg', 'sim', 'emax', 'esum', 'eavg', 'esim', 'cmax', 'csum', 'cavg', 'csim', 'remax', 'resum', 'reavg', 'resim', 'query_l']
x_smf = ['w2v_early_fusion', 'w2v_late_fusion_max', 'w2v_late_fusion_avg', 'w2v_late_fusion_sum']
y_f = ['rel']

In [16]:
clf = RandomForestClassifier(n_estimators=1000,max_features=3)

In [17]:
X = pd.concat([baseline_f[x_bf],semantic_f_[x_smf]],axis=1)
y = baseline_f[y_f]

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [19]:
clf.fit(X_train,y_train)

/home/vibhav/bar/virtualenv/lib/python3.5/site-packages/ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features=3, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [20]:
clf.score(X_test,y_test)

0.8643162393162394

In [21]:
clf.predict_proba(X_test.iloc[:1])

array([[0.748, 0.13 , 0.122]])

In [27]:
clf.predict_proba(X_test)

array([[0.748     , 0.13      , 0.122     ],
       [0.93496996, 0.02802575, 0.03700429],
       [0.795     , 0.171     , 0.034     ],
       ...,
       [0.03      , 0.03      , 0.94      ],
       [0.795     , 0.191     , 0.014     ],
       [0.54497205, 0.34481056, 0.11021739]])

In [28]:
X_test

,row,col,nul,in_link,out_link,pgcount,tImp,tPF,leftColhits,SecColhits,bodyhits,PMI,qInPgTitle,qInTableTitle,yRank,csr_score,idf1,idf2,idf3,idf4,idf5,idf6,max,sum,avg,sim,emax,esum,eavg,esim,cmax,csum,cavg,csim,remax,resum,reavg,resim,query_l,w2v_early_fusion,w2v_late_fusion_max,w2v_late_fusion_avg,w2v_late_fusion_sum
2898,17,6,26,1,0,408,1.000000,0.003281,1,1,6,0.000000,0.250000,0.041667,100,1.357434e-06,16.998778,11.131864,17.262887,12.819023,17.536209,17.262887,1.000000,1.262781,0.057399,0.728977,0.600617,77.127580,0.321365,0.416358,0.000000,0.000000,0.000000,0.000000,0.042220,0.749335,0.002997,0.030780,2,0.203786,1.0,0.069659,35.804577
2976,7,2,0,42,22,21216,1.000000,0.002699,1,3,4,0.000000,0.000000,0.500000,100,1.491094e-12,31.179463,22.432044,29.409212,32.670216,29.372600,29.409212,1.000000,2.550933,0.121473,0.920280,0.859462,105.491305,0.620537,0.730193,0.000000,0.000000,0.000000,0.000000,0.070418,0.988644,0.005816,0.036838,3,0.571639,1.0,0.156132,16.393883
2130,9,6,0,163,1027,108635,0.200000,0.000632,0,0,0,-0.519924,0.000000,0.111111,100,1.650595e-09,22.340225,20.985845,25.342058,24.369904,25.788879,25.342058,1.000000,5.258633,0.194764,0.646295,0.839834,70.983811,0.415110,0.434293,0.182574,0.182574,0.000913,0.004591,0.062994,1.141201,0.005706,0.044252,3,0.559939,1.0,0.178092,80.675690
214,3,5,6,106,131,243528,1.000000,0.000174,0,0,0,-0.415888,0.500000,0.000000,100,7.785847e-08,20.058519,16.642336,21.792883,22.890487,22.183271,21.792883,1.000000,2.349545,0.130530,0.455557,0.883298,77.933984,0.708491,0.923798,0.258199,1.876622,0.017060,0.067316,0.068041,0.986445,0.008968,0.057846,2,0.399122,1.0,0.127495,5.864788
1068,22,11,135,116,131,151296,0.333333,0.002110,0,0,9,-0.225447,0.000000,0.000000,100,9.644766e-10,23.312856,18.599820,25.777226,26.981492,25.687326,25.777226,0.470856,1.356761,0.052183,0.489001,0.896800,153.402824,0.697286,0.811664,0.000000,0.000000,0.000000,0.000000,0.052414,0.439507,0.001831,0.012490,2,0.574837,1.0,0.230576,126.355583
2184,1,2,0,146,12,4275,0.166667,0.001812,0,0,0,0.000000,0.333333,0.166667,100,7.116420e-08,21.595330,16.258688,20.345690,22.661341,20.620783,20.345690,1.000000,2.577708,0.184122,0.835254,0.925247,65.054648,0.591406,0.738699,0.000000,0.000000,0.000000,0.000000,0.038125,0.135422,0.001231,0.008099,2,0.703807,1.0,0.312885,11.889642
969,65,6,0,0,0,302,1.000000,0.048872,15,0,40,-0.087281,0.000000,0.000000,100,1.794464e-11,29.237457,20.730996,27.548780,24.639063,28.296670,27.548780,0.216274,2.883957,0.060082,0.067928,0.844620,22.721723,0.568043,0.611476,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3,0.279636,1.0,0.061734,763.963013
3004,65,8,89,27,5,32908,1.000000,0.351351,0,1,61,-0.115886,0.000000,0.000000,100,3.284645e-11,31.179463,22.432044,29.409212,32.670216,29.372600,29.409212,0.405676,2.832243,0.085826,0.411923,0.949852,49.128760,0.545875,0.634698,0.000000,0.000000,0.000000,0.000000,0.150970,1.543596,0.014033,0.088557,3,0.505717,1.0,0.130368,248.351181
2944,9,6,7,44,21,1392,0.333333,0.010576,0,4,13,0.000000,0.000000,0.142857,100,2.810945e-12,25.003739,21.494719,26.620225,27.569069,26.828212,26.620225,1.000000,3.605607,0.069339,0.223302,0.617631,45.138659,0.410351,0.481224,0.000000,0.000000,0.000000,0.000000,0.024140,0.024140,0.000161,0.001636,5,0.700819,1.0,0.151432,112.059319
2507,4,3,0,41,5,5518,0.500000,0.000149,0,5,8,-0.368829,0.000000,0.000000,100,6.025741e-13,36.988598,24.941031,33.594832,24.111099,34.089695,33.594832,0.551278,3.159318,0.065819,0.325378,0.850865,62.188424,0.628166,0.789145,0.000000,0.000000,0.000000,0.000000,0.072169,0.664068,0.006037,0.042577,4,0.493417,1.0,0.089573,26.513638
